In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using KrylovKit
using SparseArrays
using LinearAlgebra
using Plots
using LaTeXStrings
using FFTW
using LsqFit

using QuantumModels
using MatrixProductStateTools
using ExactDiagonalizationTools
using SymTensors
using GaussianFermions
using GutzwillerMPS

In [ ]:
#include("/home/amir/work/mps/test/runtests.jl");

In [ ]:
# exact Heisenberg AF mps
lx = 16
H, block_states = xxz_hamiltonian(lx, 1.0, mode=:U1)
es, vs = eigsolve(H, 1, :SR, ishermitian=true)
mpsheis = U1MPState(lx, 2, vs[1]);

In [ ]:
sz, sp, sm = spinoperators(1/2, symmetry=:U1)
heis_pm = measure(mpsheis, sp, sm)
heis_mp = measure(mpsheis, sm, sp)
heis_zz = measure(mpsheis, sz, sz)
indexes = [(1,l+1) for l=1:lx-1]

p = plot(xaxis=(L"$x$"), yaxis=(L"$\langle s^\alpha_1 s^\beta_x\rangle$"), legend=:bottomright)
plot!(p, [2:lx], [heis_zz[i] for i in indexes], marker=(:circle), label=L"\alpha=z, \beta=z")
plot!(p, [2:lx], [heis_pm[i] for i in indexes], marker=(:circle), label=L"\alpha=+, \beta=-")
plot!(p, [2:lx], [heis_mp[i] for i in indexes], marker=(:circle), label=L"\alpha=-, \beta=+")

In [ ]:
# Fermionic hopping mps using Fishman gates
L = 1 * lx
m = 100
hamil = generatebdg(triangularhopping((1,lx), 1.0, 15., 15., 15., boundary=(:OBC, :OBC)))
cm = correlationmatrix(hamil, div(L,2))
fgs = corrmat2gmps(cm)
mps = gmps2mps(fgs, m, symmetry=:U1)

fhop_pm = measure(mps, sp, sm)
#fhop_mp = measure(mps, sm, sp)
fhop_zz = measure(mps, sz, sz)
indexes = [(1,l+1) for l=1:lx-1]

p = plot(xaxis=(L"$x$"), yaxis=(L"$\langle s^\alpha_1 s^\beta_x\rangle$"), legend=:bottomright)
plot!(p, [2:lx], [fhop_zz[i] for i in indexes], marker=(:circle), label=L"\alpha=z, \beta=z")
plot!(p, [2:lx], [fhop_pm[i] for i in indexes], marker=(:circle), label=L"\alpha=+, \beta=-")
#plot!(p, [2:lx], [fhop_mp[i] for i in indexes], marker=(:circle), label=L"\alpha=-, \beta=+")

In [ ]:
# zip and gutzwiller method
mpsgutz = zipandgutzwiller!(mps, mps, maxdim=10);
@show norm(mpsgutz)
normalize!(mpsgutz);
@show norm(mpsgutz)

In [ ]:
entgutz = entanglemententropy(mpsgutz);
entheis = entanglemententropy(mpsheis);

#@show mps_dims_are_consistent(mpsgutz)
zzdata = (measure(mpsgutz, sz, sz))
pmdata = (measure(mpsgutz, sp, sm)) 
mpdata = (measure(mpsgutz, sm, sp)) 

pR = plot(xaxis=(L"$x$"), yaxis=(L"$\langle \mathbf{S}_1 \cdot \mathbf{S}_x\rangle$"), legend=false)
pF = plot(
    xaxis=(L"q", (0, pi), ([0,pi/2,pi], [L"0", L"\pi/2", L"\pi"])),
    yaxis=(L"\langle \mathbf{S}_q \cdot \mathbf{S}_{-q}\rangle"),
    legend=false
)

indexes = [(1,l+1) for l=1:lx-1]
data = [zzdata[i] for i in indexes] + [pmdata[i] for i in indexes]
data = vcat([3/4.], data)

#plot!(pR, [1:lx], data, marker=(:circle))
plot!(pR, [2:lx], [zzdata[i] for i in indexes], marker=(:circle))
plot!(pR, [2:lx], [pmdata[i] for i in indexes], marker=(:circle))
plot!(pR, [2:lx], [mpdata[i] for i in indexes], marker=(:circle))

fdata = rfft(data)
qs = range(0, pi, length=div(lx,2)+1)
plot!(pF, qs, real.(fdata))
    
plot(pF, pR, size=(800,400), layout=(1,2))

In [ ]:
entgutz = entanglemententropy(mpsgutz);
entheis = entanglemententropy(mpsheis);

In [ ]:
@. fitfn(x, p) = p[1]/6 * log(lx/pi * sin(pi*x/lx)) + p[2]
fitgutz = curve_fit(fitfn, collect(1:lx-1), entgutz, [1., 0.5])
csgutz = coef(fitgutz)
println("Gutzwiller fit", csgutz)

fitheis = curve_fit(fitfn, collect(1:lx-1), entheis, [1., 0.5])
csheis = coef(fitheis)
println("Heisenberg fit", csheis)
xsfill = range(1, lx-1, length=100)

p = plot(
    xaxis=(L"\ell"),
    yaxis=(L"S_1(\ell)"),
    legend=:bottom
)
plot!(p, entgutz, label="Gutz")
plot!(p, xsfill, fitfn(xsfill, csgutz), label=latexstring("c = $(csgutz[1])"))

plot!(p, entheis, label="Heis")
plot!(p, xsfill, fitfn(xsfill, csheis), label=latexstring("c = $(csheis[1])"))